In [3]:
import requests
from requests import Response
url = "https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json"
try:
    response:Response = requests.get(url)
    data = response.json()
    print(response.text)
except Exception as e:
    print(e)


{"type": "FeatureCollection","features": [
{"type": "Feature",	"geometry":{"type": "Polygon",	"coordinates": [[[305648.997,2768207.525],[305648.972,2768207.501],[305630.642,2768189.685],[305626.985,2768186.131],[305625.246,2768184.441],[305622.284,2768181.562],[305619.282,2768178.644],[305614.733,2768174.223],[305603.969,2768163.762],[305600.756,2768160.631],[305595.152,2768155.192],[305590.604,2768150.77],[305584.973,2768145.298],[305583.79,2768144.148],[305580.285,2768140.741],[305584.946,2768136.475],[305624.782,2768175.194],[305624.557,2768175.6],[305644.009,2768194.454],[305677.639,2768227.022],[305679.244,2768228.31],[305680.607,2768228.661],[305682.096,2768228.088],[305682.694,2768227.658],[305682.711,2768227.675],[305676.011,2768233.78],[305667.218,2768225.235],[305659.151,2768217.394],[305656.944,2768215.249],[305653.509,2768211.911],[305650.023,2768208.522],[305649.982,2768208.482],[305648.997,2768207.525],[305648.997,2768207.525]]] } ,	"properties": {	"BILL_CODE": "109671135

In [2]:
import requests
from requests import Response
import csv
import pandas as pd
import os
import sqlite3
from pyproj import Proj, transform

In [4]:
'''得到資料裡面的coordinates 第0個欄位是座標屬於的資料'''
result = []

# 遍歷 features 中的每個 JSON 物件
for feature in data["features"]:
    # 提取該 feature 的 geometry["0"] 中的所有座標
    coordinates_list = feature["geometry"]["coordinates"][0]
    
#     # 提取該 feature 的 BILL_CODE
    bill_code = feature["properties"]["BILL_CODE"]
    
    # # 遍歷每個座標，將 BILL_CODE 和座標結合，並加入結果列表
    for coordinates in coordinates_list:
        result.append([bill_code] + coordinates)

# 顯示結果
display(result)

[['10967113546410', 305648.997, 2768207.525],
 ['10967113546410', 305648.972, 2768207.501],
 ['10967113546410', 305630.642, 2768189.685],
 ['10967113546410', 305626.985, 2768186.131],
 ['10967113546410', 305625.246, 2768184.441],
 ['10967113546410', 305622.284, 2768181.562],
 ['10967113546410', 305619.282, 2768178.644],
 ['10967113546410', 305614.733, 2768174.223],
 ['10967113546410', 305603.969, 2768163.762],
 ['10967113546410', 305600.756, 2768160.631],
 ['10967113546410', 305595.152, 2768155.192],
 ['10967113546410', 305590.604, 2768150.77],
 ['10967113546410', 305584.973, 2768145.298],
 ['10967113546410', 305583.79, 2768144.148],
 ['10967113546410', 305580.285, 2768140.741],
 ['10967113546410', 305584.946, 2768136.475],
 ['10967113546410', 305624.782, 2768175.194],
 ['10967113546410', 305624.557, 2768175.6],
 ['10967113546410', 305644.009, 2768194.454],
 ['10967113546410', 305677.639, 2768227.022],
 ['10967113546410', 305679.244, 2768228.31],
 ['10967113546410', 305680.607, 2768228

In [123]:
coordinates_dict = {}

for item in result:
    # 取出每筆資料的編號（例如 '10967113574169'）
    identifier = item[0]
    # 取得對應的坐標資料 (x, y)
    coordinates = [item[1], item[2]]
    
    # 如果這個編號已經在字典裡，則將新的坐標資料加到現有的列表中
    if identifier not in coordinates_dict:
        coordinates_dict[identifier] = []  # 如果該編號還沒有資料，創建一個空列表
    coordinates_dict[identifier].append(coordinates)

display(coordinates_dict)

###############################################################################

# 定義 TWD97 和 WGS84 座標系統
twd97 = Proj(init='epsg:3826')  # TWD97
wgs84 = Proj(init='epsg:4326')  # WGS84

# 假設有多筆 TWD97 坐標 [[x1, y1], [x2, y2], ...]

conn = sqlite3.connect('TPEroad.db')
cursor = conn.cursor()


# 轉換所有 TWD97 坐標為 WGS84
coordinates_wgs84 = []
for key, value in coordinates_dict.items():
    coordinates_twd97 = value
    keyy = key
    for x, y in value:
        # 轉換為 WGS84 坐標
        longitude, latitude = transform(twd97, wgs84, x, y)
        coordinates_wgs84.append([longitude, latitude])
        # display(coordinates_wgs84)
        sql = '''
        INSERT INTO coordinates_test (BILL_CODE, lat, lon)
        SELECT ?, ROUND(?, 6), ROUND(?, 6)
        WHERE NOT EXISTS (
            SELECT 1 FROM coordinates_test WHERE lat = ROUND(?, 6)
        );
        '''
        cursor.execute(sql, (keyy, latitude, longitude, latitude))

conn.commit()
cursor.close()
conn.close()


{'10967113574169': [[308552.16, 2773353.307],
  [308546.447, 2773362.631],
  [308545.636, 2773362.174],
  [308539.956, 2773359.086],
  [308517.369, 2773346.862],
  [308510.022, 2773342.859],
  [308497.119, 2773336.425],
  [308489.019, 2773332.423],
  [308481.476, 2773329.017],
  [308475.489, 2773326.319],
  [308469.678, 2773324.052],
  [308453.346, 2773317.682],
  [308451.226, 2773316.857],
  [308446.581, 2773315.169],
  [308439.543, 2773312.984],
  [308436.822, 2773312.272],
  [308424.699, 2773309.131],
  [308421.539, 2773308.3],
  [308415.129, 2773306.616],
  [308409.505, 2773305.287],
  [308404.667, 2773304.15],
  [308393.057, 2773301.279],
  [308380.613, 2773298.3],
  [308390.234, 2773282.476],
  [308393.066, 2773283.682],
  [308405.821, 2773288.708],
  [308409.715, 2773290.256],
  [308426.728, 2773297.257],
  [308435.25, 2773299.903],
  [308440.96, 2773301.685],
  [308443.217, 2773302.485],
  [308465.978, 2773310.553],
  [308487.85, 2773319.679],
  [308503.874, 2773327.261],
  [30

c:\Users\ASUS\miniconda3\envs\venv1\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\ASUS\miniconda3\envs\venv1\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_23684\2176690072.py:35: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-py

In [5]:

# 定義 TWD97 和 WGS84 座標系統
twd97 = Proj(init='epsg:3826')  # TWD97
wgs84 = Proj(init='epsg:4326')  # WGS84

# 假設有多筆 TWD97 坐標 [[x1, y1], [x2, y2], ...]
coordinates_twd97 = result[0]

# 轉換所有 TWD97 坐標為 WGS84
coordinates_wgs84 = []

for x, y in coordinates_twd97:
    # 轉換為 WGS84 坐標
    longitude, latitude = transform(twd97, wgs84, x, y)
    coordinates_wgs84.append([longitude, latitude])

# 輸出轉換後的所有 WGS84 坐標
for i, (lon, lat) in enumerate(coordinates_wgs84):
    print(f"Point {i+1}: Latitude={lon}, Longitude={lat}")


c:\Users\user\miniconda3\envs\venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\miniconda3\envs\venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


ValueError: too many values to unpack (expected 2)

In [ ]:
from pyproj import Transformer

# 創建轉換器
transformer = Transformer.from_crs("epsg:3826", "epsg:4326")
twd97_x = 305689.109
twd97_y = 2768207.525
# 轉換 TWD97 到 WGS84
latitude,longitude = transformer.transform(twd97_x, twd97_y)

print(f'TWD97轉WGS84: Latitude: {latitude:.7}, Longitude: {longitude:.7}')


TWD97轉WGS84: Latitude: 25.02091, Longitude: 121.5518


twd97是台灣的座標系統
WGS84是世界的座標系統
經度 longitude
緯度 latitude
所以台灣的X = longitude, Y = latitude

In [14]:
from geopy.geocoders import Nominatim

# 初始化 Nominatim 反向地理編碼器
geolocator = Nominatim(user_agent='PROJ')

def get_address_from_coordinates(lat, lon):
    """ 根據經緯度獲取地址或道路名稱 """
    location = geolocator.reverse((lat, lon), language='zh-TW', exactly_one=True)
    if location:
        return location.address
    return "未找到地址"

# 範例：查詢經緯度 (台北市)
latitude = 25.02066
longitude = 121.5518

address = get_address_from_coordinates(latitude, longitude)
print(address)
print(type(address))
print(f"座標 ({latitude}, {longitude}) 對應的地址是：{address}")


7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣
<class 'str'>
座標 (25.02066, 121.5518) 對應的地址是：7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣


In [47]:
address = "7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣"
list_address = [item.strip() for item in address.split(",")]
list_address.reverse()
print(list_address)
dic_address = {} 


['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']


In [3]:
    
def download_data():
    '''
    下載資料 需要做驗證只寫入不在資料庫裏面的那筆
    '''
    conn = sqlite3.connect("./TPEroad.db")
    url = 'https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        with conn:
            cursor = conn.cursor()
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS records (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                Bill_code TEXT,
                RCVdate TEXT,
                Start_date TEXT,
                End_date TEXT,
                Address1 TEXT,
                X1 TEXT,
                Y1 TEXT,
                新地址 TEXT,
                行政區 TEXT,
                Lat TEXT,
                Lon TEXT,
                申請日期 TEXT,
                開始日期 TEXT,
                結束日期 TEXT,
                UNIQUE(Address1,RCVdate)
            )
            ''')
            print("Table 'records' created or already exists.")
    except Exception as e:
        print(e)

    else:
        with conn:
            cursor = conn.cursor()
            for i in data['features']:
                for k in i['properties']:
                    Bill_code:str = i['properties']["BILL_CODE"]
                    RCVdate = i['properties']["URGENT_RCV_DATE"]
                    Start_date = i['properties']["URGENT_START_DATE"]
                    End_date = i['properties']["URGENT_END_DATE"]
                    Address1 = i['properties']["URGENT_ADDRESS1"]
                    X1 = i['properties']["X1"]
                    Y1 = i['properties']["Y1"]

                    
                    cursor = conn.cursor()
                    sql = '''INSERT OR IGNORE INTO records(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1)
                                values (?, ?, ?, ?, ?, ?, ?);
                            '''
                    cursor.execute(sql,(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1))
download_data()

Table 'records' created or already exists.


現在有每一個x,y轉換成的地址 再把地址存成我用的資料表 再從資料表裡面提取我需要的區的位置 就可以做combobox的下拉式選單

In [7]:
import functoins
import sqlite3

conn = sqlite3.connect('TPEroad.db')
with conn:
    cursor = conn.cursor()

    sql = '''
        SELECT Address1,x1,y1 FROM records
    '''

    cursor.execute(sql)
    # 把地址跟x1y1座標存進一個字典裡面
    address = [ ]
    for i in cursor.fetchall():
        di = {}
        di["地址"] = i[0]
        di["x1"] = i[1]
        di["y1"] = i[2]
        address.append(di)
    cursor.close() #把cursor關掉 清出記憶體
    display(address)

[{'地址': '台北市樂業街108巷1至11號', 'x1': '305689.109', 'y1': '2768179.406'},
 {'地址': '信義區松壽路１２號  松智路17號', 'x1': '307265.536', 'y1': '2769857.038'},
 {'地址': '信義區松智路１７號 松壽路12號', 'x1': '307262.652', 'y1': '2769660.839'},
 {'地址': '中正區衡陽路１０５號', 'x1': '301411.125', 'y1': '2770545.725'},
 {'地址': '臺北市大安區信義路三段41之3號（信義辦公大樓）', 'x1': '304074.549', 'y1': '2769606.348'},
 {'地址': '中正區中華路一段５５號', 'x1': '301326.715', 'y1': '2770629.514'},
 {'地址': '中山區南京西路１５號', 'x1': '302616.632', 'y1': '2771686.983'},
 {'地址': '萬華區昆明街１０５號', 'x1': '300912.821', 'y1': '2770480.504'},
 {'地址': '中山區南京東路一段2號', 'x1': '302824.337', 'y1': '2771659.17'},
 {'地址': '大安區仁愛路四段１１８號', 'x1': '305525.047', 'y1': '2770056.092'},
 {'地址': '內湖區行善路３１號至行善路３７號', 'x1': '308220.149', 'y1': '2772242.502'},
 {'地址': '文山區忠順街二段１６號', 'x1': '306806.413', 'y1': '2764192.064'},
 {'地址': '北投區石牌路一段２００號', 'x1': '301922.631', 'y1': '2778639.744'},
 {'地址': '萬華區貴陽街二段210至230號', 'x1': '300312.125', 'y1': '2770302.614'},
 {'地址': '中正區延平南路１９７號2樓', 'x1': '301203.961', 'y1': '27

接下來要提取每個字典裏面的x1y1然後丟到我的latlonturn裡面轉成地址(['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街', '114號'])  
再for i in list if '區' in i sql='insert ? into records' execute(sql,(i))


In [12]:
import functoins
import sqlite3
import time
conn = sqlite3.connect("TPEroad.db")
cursor = conn.cursor()
alladdress = []
for i in address:
    ad = i['地址']
    fuad = functoins.latlonturn("reverse",alladdress,i['x1'],i['y1'])
    fullstreet = ''.join(fuad)
    lat,lon = functoins.xytransform(i['x1'],i['y1'])
    for i in fuad:
        if "區" in i :dist = i
    add_content_sql = '''
    UPDATE RECORDS
    SET 新地址 = ?, 行政區 = ?, Lat = ?, Lon = ?
    WHERE address1 = ?;
    '''

    format_date='''
    UPDATE records 
	SET 日期=  (SELECT strftime('%Y-%m-%d', 
	              substr(RCVdate , 1, 4) || '-' || substr(RCVdate , 5, 2) || '-' || substr(RCVdate , 7, 2)) )
    '''


    cursor.execute(add_content_sql,(fullstreet, dist, lat, lon, ad))
    time.sleep(10)
    conn.commit()


1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.02066, Longitude: 121.5518
3.單次轉換結果['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']
4.總清單:[['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']]
2.TWD97轉WGS84: Latitude: 25.02066, Longitude: 121.5518
1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.03575, Longitude: 121.5675
3.單次轉換結果['臺灣', '110062', '臺北市', '信義商圈', '信義區', '西村里', '松壽路20巷']
4.總清單:[['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號'], ['臺灣', '110062', '臺北市', '信義商圈', '信義區', '西村里', '松壽路20巷']]
2.TWD97轉WGS84: Latitude: 25.03575, Longitude: 121.5675
1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.03398, Longitude: 121.5674
3.單次轉換結果['臺灣', '110', '臺北市', '信義商圈', '信義區', '西村里', '松廉路', '76號']
4.總清單:[['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號'], ['臺灣', '110062', '臺北市', '信義商圈', '信義區', '西村里', '松壽路20巷'], ['臺灣', '110', '臺北市', '信義商圈', '信義區', '西村里', '松廉路', '76號']]
2.TWD97轉WGS84: Latitude: 25.03398, Longitude: 121.5674
1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.04218, Longitude: 121.5095
3.單次轉換結果['

KeyboardInterrupt: 

In [6]:
import functoins
address = [{'地址': '台北市樂業街108巷1至11號', 'x1': '305689.109', 'y1': '2768179.406'},
 {'地址': '士林區大東路１７６號', 'x1': '302989.777', 'y1': '2776097.833'}]
alladdress = []
# for i in address:
#     ad = i['地址']
#     fuad = functoins.latlonturn("reverse",alladdress,i['x1'],i['y1'])
#     type(fuad)
# for i in fuad:
#     if "區" in i :dist = i
#     print(dist)

fuad = functoins.latlonturn('reverse', alladdress, address[0]["x1"], address[0]["y1"])
fullstreet = ''.join(fuad)
display(fullstreet)

1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.02066, Longitude: 121.5518
3.單次轉換結果['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']
4.總清單:[['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']]


'臺灣106臺北市六張犁大安區芳和里樂業街118巷7號'

In [4]:
import sqlite3
def get_district()->list[str]:
    '''
    docString
    parameter:
    return:
        傳出所有的行政區名稱
    '''
    conn = sqlite3.connect("TPEroad.db")
    with conn:
        # Create a cursor object to execute SQL commands
        cursor = conn.cursor()
        # SQL query to select unique sitenames from records table
        sql = '''
        SELECT DISTINCT 行政區
        FROM records
        '''
        # Execute the SQL query
        cursor.execute(sql)
        # Get all results and extract first item from each row into a list
        counties = [items[0] for items in cursor.fetchall()]
    
    # Return the list of unique sitenames
    return counties
get_district()

['北投區',
 '大安區',
 '松山區',
 '士林區',
 '內湖區',
 '文山區',
 '南港區',
 '中山區',
 '大同區',
 '萬華區',
 '中正區',
 '信義區']

從資料庫拿coordinates的座標畫在mapview上面

In [3]:
bill_code = '10967113575858'
conn = sqlite3.connect('TPEroad.db')
with conn:
    cursor = conn.cursor()
    sql = '''
        SELECT distinct lat,lon
        FROM coordinates_test
        where BILL_CODE =?
        '''
    # Execute the SQL query
    cursor.execute(sql, (bill_code,))

    result = cursor.fetchall()
    coordinate_list = [[lat, lon] for lat, lon in result]


    print(coordinate_list)
    

[[25.041117, 121.52214], [25.041047, 121.522122], [25.041882, 121.518678], [25.041965, 121.518709], [25.041947, 121.518717], [25.041938, 121.518725], [25.041933, 121.51874], [25.041837, 121.519107], [25.041804, 121.519247], [25.041846, 121.519262], [25.041831, 121.519326], [25.041789, 121.519314], [25.041745, 121.519521], [25.041786, 121.519533], [25.04177, 121.519595], [25.041732, 121.519586], [25.04173, 121.519586], [25.041726, 121.519588], [25.04151, 121.520504], [25.041544, 121.520513], [25.041524, 121.520601], [25.041489, 121.520591], [25.041307, 121.521339], [25.041121, 121.522117], [25.04098, 121.522105], [25.041063, 121.521767], [25.041225, 121.521088], [25.041381, 121.520435], [25.041426, 121.520245], [25.041516, 121.51987], [25.041538, 121.519785], [25.041659, 121.519291], [25.041679, 121.51921], [25.041727, 121.519013], [25.041686, 121.519003], [25.041719, 121.518861], [25.041758, 121.518873], [25.041798, 121.518713], [25.0418, 121.51869], [25.041799, 121.518677], [25.041797

tkmap做兩點距離或範圍

In [4]:
import tkinter as tk
from tkinter import ttk
import tkintermapview

# 創建 Tkinter 主窗口
root = tk.Tk()
root.title("./tkintermapview Example")

# 設置地圖視窗
map_frame = ttk.Frame(root)
map_widget = tkintermapview.TkinterMapView(map_frame, width=600, height=400, corner_radius=0)

# 設定初始地圖顯示位置（台北市）
# map_widget.set_position(25.02091606, 121.55139558, marker=True)
map_widget.set_position(25.041117, 121.52214)

# # 三個點的經緯度
points = [
    {"name": "Point 1", "lat": 25.0330, "lon": 121.5674},  # 例如台北101
    {"name": "Point 2", "lat": 25.0340, "lon": 121.5644},  # 點2
    {"name": "Point 3", "lat": 25.0320, "lon": 121.5664},  # 點3
]
ppoint = [[25.041117, 121.52214], [25.041047, 121.522122], [25.041882, 121.518678], [25.041965, 121.518709], [25.041947, 121.518717], [25.041938, 121.518725], [25.041933, 121.51874], [25.041837, 121.519107], [25.041804, 121.519247], [25.041846, 121.519262], [25.041831, 121.519326], [25.041789, 121.519314], [25.041745, 121.519521], [25.041786, 121.519533], [25.04177, 121.519595], [25.041732, 121.519586], [25.04173, 121.519586], [25.041726, 121.519588], [25.04151, 121.520504], [25.041544, 121.520513], [25.041524, 121.520601], [25.041489, 121.520591], [25.041307, 121.521339], [25.041121, 121.522117], [25.04098, 121.522105], [25.041063, 121.521767], [25.041225, 121.521088], [25.041381, 121.520435], [25.041426, 121.520245], [25.041516, 121.51987], [25.041538, 121.519785], [25.041659, 121.519291], [25.041679, 121.51921], [25.041727, 121.519013], [25.041686, 121.519003], [25.041719, 121.518861], [25.041758, 121.518873], [25.041798, 121.518713], [25.0418, 121.51869], [25.041799, 121.518677], [25.041797, 121.518665], [25.041769, 121.518639], [25.04175, 121.518629]]







# # 在地圖上添加這三個點的標記
# for point in ppoint:
#     map_widget.set_marker(point[0], point[1])

# # 計算這三個點的範圍（bounding box）
# lats = [point["lat"] for point in points]
# lons = [point["lon"] for point in points]
# min_lat, max_lat = min(lats), max(lats)
# min_lon, max_lon = min(lons), max(lons)

# # 計算範圍的中心點和放大等級
# center_lat = (min_lat + max_lat) / 2
# center_lon = (min_lon + max_lon) / 2

# # 設置地圖顯示範圍
# map_widget.set_position(center_lat, center_lon, zoom=15)

# # 畫出範圍的矩形
map_widget.set_polygon(ppoint)

# 打包顯示地圖
map_widget.pack()
map_frame.pack(padx=10, pady=10)

# 啟動 Tkinter 主循環
root.mainloop()
